In [14]:
import math
import random
import numpy

###### STEP 4 ######

#Team Draft Interleaving


#to be able to distinguish the document's assignment, rename the relevances by adding index number
#eg. Ranking pair:  [['HR', 'HR', 'R', 'R', 'N'], ['HR', 'R', 'R', 'N', 'N']]
# Updated pair : (['HR0', 'HR1', 'R2', 'R3', 'N4'], ['HR5', 'R6', 'R7', 'N8', 'N9'])

def preprocess(pair):
    i = 0
    updated_pair = ([], [])
    for relevance in pair[0]:
        updated_pair[0].append(relevance+ str(i))
        i+=1
    
    for relevance in pair[1]:
        updated_pair[1].append(relevance+ str(i))
        i+=1        
    
    return updated_pair


#a) create the interleaving list
def team_draft_interleave(pair):
    pair = preprocess(pair)
    
    A = pair[0][:]
    B = pair[1][:]
    doc_list = []  #the interleaving list
    assignments = [] #holds the pair that the documents taken from, "A or B", "0 or 1" respectively
    while A or B:
        if A and B:
            first = random.randint(0,1)
        elif A:
            first = 0
        else:
            first = 1
        second = int(math.fabs(first-1))

        #pick from the first list(A) then the second list (B)
        if first == 0: #A
            doc = A[0]
            doc_list.append(doc)
            assignments.append(first)
            #delete the doc from the first and the second list
            del A[0] 
            if doc in B:
                B.remove(doc)
            
            #pick from the second list
            doc = B[0]
            doc_list.append(doc)
            assignments.append(second) 
            #delete the doc from the second list (top element) and the first list
            del B[0]
            
            if doc in A:
                A.remove(doc)
            
       #pick from the first list(B) then the second list (A)
        else: #B
            doc = B[0]
            doc_list.append(doc)
            assignments.append(first)
            del B[0]
            if doc in A:
                A.remove(doc)

            #pick from the second list
            doc = A[0]
            doc_list.append(doc)
            assignments.append(second)
            #delete the doc from the first and the second list
            del A[0] 
            if doc in B:
                B.remove(doc)
    
       
    #append the interleaved list with the assignment        
    output = []
    for i in range(len(doc_list)):
        output.append([doc_list[i],assignments[i]])
    return output



#generate a list of 'number' amount of random clicks
def random_clicks(length, number):
    clicks = random.sample(range(length), number)
    return clicks

    
#b) evaluate the interleaved list with random clicks


def team_draft_evaluation(interleaved, clicks):
    doc_list, assignments = zip(*interleaved) 
#     print(doc_list)
#     print(assignments)
    click_nums = [0, 0]
    for i in clicks:
        click_nums[assignments[i]]+=1

    return click_nums[0]-click_nums[1]


pair = [["HR", "HR", "R", "R", "N"], ["HR", "R", "R", "N", "N"]]
print ("Ranking pair: ", pair)
updated_pair = preprocess(pair)
print("Updated pair: ", updated_pair)
interleaved = team_draft_interleave(pair)
print("interleaved: ", interleaved)

clicks = random_clicks(len(interleaved), 3)
print("Random Clicks: ", clicks)

result = team_draft_evaluation(interleaved, clicks)
print("Evaluation A-B (number of clicks): ", result)

Ranking pair:  [['HR', 'HR', 'R', 'R', 'N'], ['HR', 'R', 'R', 'N', 'N']]
Updated pair:  (['HR0', 'HR1', 'R2', 'R3', 'N4'], ['HR5', 'R6', 'R7', 'N8', 'N9'])
interleaved:  [['HR0', 0], ['HR5', 1], ['R6', 1], ['HR1', 0], ['R7', 1], ['R2', 0], ['N8', 1], ['R3', 0], ['N9', 1], ['N4', 0]]
Random Clicks:  [0, 4, 3]
Evaluation A-B (number of clicks):  1


In [51]:
### STEP 5 ####

# Yandex Click Log File
def get_sessions():
    f = open('YandexRelPredChallenge.txt', 'r')
    content = []
    for line in f:
        line = line.split()
        content.append(line)
    f.close()

    #print(content[:10])

    #each session is a query that has the list of documents retrieved and list of documents that are clicked
    sessions = []  
    for line in content:
        if(line[2]) == 'Q': 
            session = []
            session.append(line[5:])
            session.append([])
            sessions.append(session)
        if(line[2]) == 'C':
            c = line[-1] 
            for i in range(-1,-len(sessions), -1): #attribute the click to the last query that had this document as a result
                session = sessions[i]
                if c in session[0]:
                    sessions[i][1].append(c)
                    break;
    return sessions

sessions = get_sessions()
print(len(sessions))

42652


In [52]:
#Random Click Model


#parameter from the user log
def RCM_parameter(sessions):
    sum1 = 0
    sum2 = 0
    for s in sessions:
        clicks = set()
        for c in s[1]:
            clicks.add(c)
        sum1 += len(clicks)
        sum2 += len(s[0])
    
    p = 1.0*sum1/sum2 
        
    return p


def RCM_clicks(length, p):
    clicks = []
    for i in range(length):
        prob = random.uniform(0, 1)
        if prob < p:
            clicks.append(i)
    return clicks

p = RCM_parameter(sessions)
print(p)

clicks = RCM_clicks(10, p)
print(clicks)

0.12631295132701867
[0, 3, 5, 6]


In [31]:
interleaved = [['HR0', 0], ['HR5', 1], ['R6', 1], ['HR1', 0], ['R7', 1], ['R2', 0], ['N8', 1], ['R3', 0], ['N9', 1], ['N4', 0]]

relevanceScores = { 'N':0, 'R':3, 'HR':4 }


def getProbabilityOfRelevance (relevance):
    # from paper, similar to DCG score
    gain = (2 ** relevanceScores[relevance]) - 1
    discount = 2 ** max(relevanceScores.values())
    probability = gain / discount
    return probability

def attractiveness(interleaved):
    attr = []
    for doc in interleaved:
        relevance = doc[0][0]
        if relevance == 'H':
            attr.append(getProbabilityOfRelevance ("HR"))
        if relevance == 'R':
            attr.append(getProbabilityOfRelevance ("R"))
        if relevance == 'N':
            attr.append(getProbabilityOfRelevance ("N"))
    return attr
    


def lambda_r(sessions, r):
    S = list(filter(lambda s: s[0][r-1] in s[1] , sessions))
    total = 0
    for s in S:
        click_index = [s[0].index(page) for page in s[1]]
        last_click = click_index[-1] + 1 #satisfied
        if last_click != r:
            total += 1

    return total/float(len(S))  
    

def get_lambdas(sessions):
    lambdas = []
    for i in range(10):
        lambdas.append(lambda_r(sessions, i+1))
    return lambdas
    

def SDCM_clicks(interleaved, lambdas):
    clicks = []
    attr = attractiveness(interleaved)
    n = len(attr)
    for i in range(1, n):
        prob = random.uniform(0,1)
        
        if prob < attr[i]: #clicked
            clicks.append(i)
            prob = random.uniform(0,1)
            if prob > lambdas[i]: #stop
                break
                
    return clicks


In [54]:
###### STEP 6 #####

# For simulation, we need to run the following piece of code for each pair to get the score
# result : which search engine has higher clicks, A - B, eg. if it is higher than 0, the first one wins
# each run has a different result, so 

pair = [["HR", "HR", "R", "R", "N"], ["HR", "R", "R", "N", "N"]]
sessions = get_sessions()   #gets the sessions as a list from yandex log file
p = RCM_parameter(sessions) #calculates the RCM parameter, MLE
lambdas = get_lambdas(sessions)  #calculates all the lambda values for each rank (1-10)
print(lambdas)

def simulation_for_pair(pair, click_model):
    interleaved = team_draft_interleave(pair)  #gets the interleaved list

    clicks_RCM = RCM_clicks(10, p)  #generates clicks based on RCM model

    clicks_SDCM = SDCM_clicks(interleaved, lambdas) #generates clicks based on RCM model

    if click_model == "RCM":
        result = team_draft_evaluation(interleaved, clicks_RCM)
    if click_model == "SDCM":
        result = team_draft_evaluation(interleaved, clicks_SDCM)
    
    return result


def N_simulation(pair, click_model, N):
    A = 0 #first search engine score
    B = 0 #second search engine score
    
    for i in range(N):
        result = simulation_for_pair(pair, click_model)
        if result > 0:
            A+=1
        if result < 0:
            B+=1
    return A, B

A_score, B_score = N_simulation(pair, "RCM", 100)
print(A_score, B_score)


[0.31942931258106355, 0.5378201310303752, 0.5706431189603466, 0.58397365532382, 0.5779661016949152, 0.5579487179487179, 0.5545207605743112, 0.5058619192357794, 0.468378506894912, 0.22727272727272727]
32 42
